In [ ]:
!rm -rf sample_data

# Conversion del comprimido en logs

In [ ]:
import json
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [ ]:
DATA_DIR = Path('./data').resolve()
assert DATA_DIR.exists()

Recuperado de: *back/core/processing.py*

In [ ]:
# Similar a read_json_course
# Leemos el json y lo transformamos en dataframe
sample_json = json.loads((DATA_DIR / "sample.json").read_text())
sample = pd.DataFrame.from_records(sample_json)
sample

,accept_language,agent,context,event,event_source,event_type,host,page,referer,session,time,username
67243,,,"{'user_id': None, 'path': '/edxucursos/externo...","{""GET"": {""ticket"": [""095b5cbeaf0f5fafbd1a7c67e...",server,/edxucursos/externo,eol.uchile.cl,None,,,2023-04-03T15:19:49.724704+00:00,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...
138051,"es-ES,es;q=0.9",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,"{'user_id': 86459, 'path': '/event', 'course_i...","{""id"": ""1929ac93da0340fb82a1ab1c4ead8b16"", ""co...",browser,load_video,eol.uchile.cl,https://eol.uchile.cl/courses/course-v1:eol+FA...,https://eol.uchile.cl/courses/course-v1:eol+FA...,593c582e46365b3f0131a6ba45c1f020,2023-04-03T23:33:22.505486+00:00,ed31b867c1124ff082e52fbaba03e11865e4da8e38bed1...
91742,,,"{'user_id': None, 'path': '/edxucursos/externo...","{""GET"": {""ticket"": [""83f050b3e7fd35c3f8d5d351c...",server,/edxucursos/externo,eol.uchile.cl,None,,,2023-04-03T17:08:17.731079+00:00,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...
59930,"es-419,es;q=0.9",Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,"{'user_id': 10664, 'path': '/event', 'course_i...",input_48405adfc393404187ea34ddc4b8cf99_2_1=Falso,browser,problem_check,eol.uchile.cl,https://eol.uchile.cl/courses/course-v1:eol+FC...,https://eol.uchile.cl/courses/course-v1:eol+FC...,3a83e338ad65749b220bc68ff3d6db64,2023-04-03T14:25:58.961664+00:00,ef260ef56ad2a0094843cc758b746a71a531f6f15fabd9...
1071,"es-US,es-419;q=0.9,es;q=0.8,en;q=0.7",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,"{'user_id': 28583, 'path': '/event', 'course_i...","{""id"": ""eaa4df4f12c8417b9c06305832d1675d"", ""co...",browser,play_video,eol.uchile.cl,https://eol.uchile.cl/courses/course-v1:eol+IN...,https://eol.uchile.cl/courses/course-v1:eol+IN...,5d38ae6950793cc1b737403273088b9b,2023-04-03T00:04:13.823382+00:00,e26d1d67c708b50eaaa6a48cda1e47307d457a5fb591cd...
...,...,...,...,...,...,...,...,...,...,...,...,...
112645,"es-ES,es;q=0.9",Mozilla/5.0 (iPhone; CPU iPhone OS 16_4 like M...,{'course_id': 'course-v1:eol+FACSO-TDA+2022_1'...,"{""GET"": {""fbclid"": [""PAAaYU3gyLJWd3YxFdjJ8MzIC...",server,/courses/course-v1:eol+FACSO-TDA+2022_1/about,eol.uchile.cl,None,https://l.instagram.com/,,2023-04-03T20:25:51.455901+00:00,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...
65702,"es-ES,es;q=0.9",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,"{'course_id': 'course-v1:eol+ENNEG388+2023_1',...","{""GET"": {}, ""POST"": {}}",server,/courses/course-v1:eol+ENNEG388+2023_1/course/,eol.uchile.cl,None,https://eol.uchile.cl/courses/course-v1:eol+EN...,d33061f84a2a20221095cb750872bc1c,2023-04-03T15:04:37.771011+00:00,43f74274c312f52f5165bb627a7c85e95fd0c7ae9453aa...
130881,,,"{'course_id': 'course-v1:eol+FEN-MO+2023_1', '...","{'user_id': '10980', 'course_id': 'course-v1:e...",server,edx.grades.subsection.grade_calculated,,None,,,2023-04-03T22:38:59.936112+00:00,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...
133574,,,"{'course_id': 'course-v1:eol+IN2201+2023_1', '...","{'user_id': '46651', 'course_id': 'course-v1:e...",server,edx.grades.subsection.grade_calculated,,None,,,2023-04-03T23:10:31.504335+00:00,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...


In [ ]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 67243 to 64515
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   accept_language  300 non-null    object
 1   agent            300 non-null    object
 2   context          300 non-null    object
 3   event            300 non-null    object
 4   event_source     300 non-null    object
 5   event_type       300 non-null    object
 6   host             300 non-null    object
 7   page             82 non-null     object
 8   referer          300 non-null    object
 9   session          300 non-null    object
 10  time             300 non-null    object
 11  username         300 non-null    object
dtypes: object(12)
memory usage: 38.6+ KB


In [ ]:
# Funcion auxiliar que expande la columna context en
# course_id, org_id, path y user_id

# Nota: El compressed sample no trae name ni ip debido a la anonimizacion
def read_logs(df):
    """ Read logs and expand inner JSON values

    Recover valuable info

    Arguments:
        dataframe Pandas DataFrame
    Returns:
        expanded_records List of dictionnaries
    """
    def add_if_available(row, field, subfield):
        try:
            return row[field][subfield]
        except Exception:
            return np.nan
    df["context.course_id"] = df.apply(
        lambda x: add_if_available(x, "context", "course_id"), axis=1)
    df["context.org_id"] = df.apply(
        lambda x: add_if_available(x, "context", "org_id"), axis=1)
    df["context.path"] = df.apply(
        lambda x: add_if_available(x, "context", "path"), axis=1)
    df["context.user_id"] = df.apply(
        lambda x: add_if_available(x, "context", "user_id"), axis=1)
    # Note: extra values to be extracted should be added here
    return df

In [ ]:
data_df = read_logs(sample) # Obtencion de los logs
log_df = data_df[data_df.apply(lambda row: row["username"] != '', axis=1)] # limpiamos
log_df.info()
log_df

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 67243 to 64515
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   accept_language    300 non-null    object 
 1   agent              300 non-null    object 
 2   context            300 non-null    object 
 3   event              300 non-null    object 
 4   event_source       300 non-null    object 
 5   event_type         300 non-null    object 
 6   host               300 non-null    object 
 7   page               82 non-null     object 
 8   referer            300 non-null    object 
 9   session            300 non-null    object 
 10  time               300 non-null    object 
 11  username           300 non-null    object 
 12  context.course_id  300 non-null    object 
 13  context.org_id     300 non-null    object 
 14  context.path       254 non-null    object 
 15  context.user_id    205 non-null    float64
dtypes: float64(1), object(15)

,accept_language,agent,context,event,event_source,event_type,host,page,referer,session,time,username,context.course_id,context.org_id,context.path,context.user_id
67243,,,"{'user_id': None, 'path': '/edxucursos/externo...","{""GET"": {""ticket"": [""095b5cbeaf0f5fafbd1a7c67e...",server,/edxucursos/externo,eol.uchile.cl,None,,,2023-04-03T15:19:49.724704+00:00,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,,,/edxucursos/externo,NaN
138051,"es-ES,es;q=0.9",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,"{'user_id': 86459, 'path': '/event', 'course_i...","{""id"": ""1929ac93da0340fb82a1ab1c4ead8b16"", ""co...",browser,load_video,eol.uchile.cl,https://eol.uchile.cl/courses/course-v1:eol+FA...,https://eol.uchile.cl/courses/course-v1:eol+FA...,593c582e46365b3f0131a6ba45c1f020,2023-04-03T23:33:22.505486+00:00,ed31b867c1124ff082e52fbaba03e11865e4da8e38bed1...,course-v1:eol+FACSO-MIS+2023_1a,eol,/event,86459.0
91742,,,"{'user_id': None, 'path': '/edxucursos/externo...","{""GET"": {""ticket"": [""83f050b3e7fd35c3f8d5d351c...",server,/edxucursos/externo,eol.uchile.cl,None,,,2023-04-03T17:08:17.731079+00:00,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,,,/edxucursos/externo,NaN
59930,"es-419,es;q=0.9",Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6...,"{'user_id': 10664, 'path': '/event', 'course_i...",input_48405adfc393404187ea34ddc4b8cf99_2_1=Falso,browser,problem_check,eol.uchile.cl,https://eol.uchile.cl/courses/course-v1:eol+FC...,https://eol.uchile.cl/courses/course-v1:eol+FC...,3a83e338ad65749b220bc68ff3d6db64,2023-04-03T14:25:58.961664+00:00,ef260ef56ad2a0094843cc758b746a71a531f6f15fabd9...,course-v1:eol+FCFM-APEG+2023_1,eol,/event,10664.0
1071,"es-US,es-419;q=0.9,es;q=0.8,en;q=0.7",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,"{'user_id': 28583, 'path': '/event', 'course_i...","{""id"": ""eaa4df4f12c8417b9c06305832d1675d"", ""co...",browser,play_video,eol.uchile.cl,https://eol.uchile.cl/courses/course-v1:eol+IN...,https://eol.uchile.cl/courses/course-v1:eol+IN...,5d38ae6950793cc1b737403273088b9b,2023-04-03T00:04:13.823382+00:00,e26d1d67c708b50eaaa6a48cda1e47307d457a5fb591cd...,course-v1:eol+IN2201+2023_1,eol,/event,28583.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,"es-ES,es;q=0.9",Mozilla/5.0 (iPhone; CPU iPhone OS 16_4 like M...,{'course_id': 'course-v1:eol+FACSO-TDA+2022_1'...,"{""GET"": {""fbclid"": [""PAAaYU3gyLJWd3YxFdjJ8MzIC...",server,/courses/course-v1:eol+FACSO-TDA+2022_1/about,eol.uchile.cl,None,https://l.instagram.com/,,2023-04-03T20:25:51.455901+00:00,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,course-v1:eol+FACSO-TDA+2022_1,eol,/courses/course-v1:eol+FACSO-TDA+2022_1/about,NaN
65702,"es-ES,es;q=0.9",Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,"{'course_id': 'course-v1:eol+ENNEG388+2023_1',...","{""GET"": {}, ""POST"": {}}",server,/courses/course-v1:eol+ENNEG388+2023_1/course/,eol.uchile.cl,None,https://eol.uchile.cl/courses/course-v1:eol+EN...,d33061f84a2a20221095cb750872bc1c,2023-04-03T15:04:37.771011+00:00,43f74274c312f52f5165bb627a7c85e95fd0c7ae9453aa...,course-v1:eol+ENNEG388+2023_1,eol,/courses/course-v1:eol+ENNEG388+2023_1/course/,26926.0
130881,,,"{'course_id': 'course-v1:eol+FEN-MO+2023_1', '...","{'user_id': '10980', 'course_id': 'course-v1:e...",server,edx.grades.subsection.grade_calculated,,None,,,2023-04-03T22:38:59.936112+00:00,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,course-v1:eol+FEN-MO+2023_1,eol,NaN,NaN
133574,,,"{'course_id': 'course-v1:eol+IN2201+2023_1', '...","{'user_id': '46651', 'course_id': 'course-v1:e...",server,edx.grades.subsection.grade_calculated,,None,,,2023-04-03T23:10:31.504335+00:00,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,course-v1:eol+IN2201+2023_1,eol,NaN,NaN


# Obtencion de las métricas a partir de los logs
1. Videos vistos por los usuarios (para esta métrica primero es necesaria la obtención del id de los videos y los segmentos de video vistos/adelantados/repetidos. Los segmentos pueden ser utiles para saber mas sobre el comportamiento del usuario)
2. TODO: Cobertura de videos por usuario (para esta métrica se hace uso de los segmentos, intersectandolos para calcular la cobertura total sobre el video)
3. TODO: Tiempo en un documento
4. TODO: Visitas a un documento
5. TODO: Completitud de una unidad (esta estadística lista a los usuarios que han completado una unidad vs quienes aún no la completan)

# Videos vistos por los usuarios


Recuperado de: *back/views/tasks.py*

In [ ]:
# Funciones auxiliares para process_log_views
def get_video_logs(course_data):
        """
        From course logs, returns video type logs like play_video or stop_video
        """
        video_event_type = [
            'hide_transcript',
            'load_video',
            'pause_video',
            'play_video',
            'seek_video',
            'show_transcript',
            'speed_change_video',
            'stop_video',
            'video_hide_cc_menu',
            'video_show_cc_menu',
        ]
        video_mobile_type = [
            'edx.video.transcript.hidden',
            'edx.video.loaded',
            'edx.video.paused',
            'edx.video.played',
            'edx.video.position.changed',
            'edx.video.transcript.shown',
            'edx.video.stopped',
        ]
        raw_video_course_logs = course_data[course_data.event_type.isin(
            video_event_type + video_mobile_type)]
        return raw_video_course_logs

In [ ]:
video_logs_df = get_video_logs(log_df) # logs filtrados relacionados a videos

In [ ]:
def make_segment_dataframe(grouped):
        """
        Creates a dataframe with start-stop segment per user and video
        """
        df_cols = ['id', 'username', 'time', 'start', 'end']
        all_pairs = []
        malformed_pairs = 0
        for name, group in grouped:
            state = {'id': None, 'init': None, 'action': None}
            for index, row in group.iterrows():
                etype = row.event_type
                if etype == 'play_video':
                    state['id'] = row.id
                    state['init'] = row.currenttime
                    state['action'] = row.event_type
                elif etype == 'seek_forward':
                    if state['id'] == row.id:
                        # skip segment
                        # all_pairs.append
                        if state['action'] == 'play_video':
                            try:
                                assert (row.old <= row.new)
                                all_pairs.append((row.id, row.username,
                                                  row.time, row.old, row.new))
                            except AssertionError:
                                malformed_pairs += 1

                    state['id'] = row.id
                    state['init'] = row.currenttime
                    state['action'] = row.event_type
                elif etype == 'seek_back':
                    if state['id'] == row.id and state[
                            'action'] == 'play_video':
                        try:
                            assert (state['init'] <= row.old)
                            all_pairs.append((row.id, row.username, row.time,
                                              state['init'], row.old))
                        except AssertionError:
                            malformed_pairs += 1

                    state['id'] = row.id
                    state['init'] = row.currenttime
                    state['action'] = row.event_type
                elif etype == 'pause_video' or etype == 'stop_video':
                    if state['id'] == row.id and state[
                            'action'] == 'play_video':
                        try:
                            assert (state['init'] <= row.currenttime)
                            all_pairs.append((row.id, row.username, row.time,
                                              state['init'], row.currenttime))
                        except AssertionError:
                            malformed_pairs += 1
                    state['id'] = row.id
                    state['init'] = row.currenttime
                    state['action'] = row.event_type
                else:
                    continue
        print("Malformed pairs processed: {}".format(malformed_pairs))
        segments_df = pd.DataFrame(all_pairs, columns=df_cols)
        return segments_df

Recuperado de: *back/views/processing.py*

In [ ]:
# Funciones auxiliares para process_views
pps_videore = re.compile(
    r'((?<="duration": )[0-9.]+?(?=,|}))|((?<="code": ").*?(?="))|((?<="id": ").*?(?="))|((?<="currentTime": )[0-9.]+?(?=,|}))'
)
load_videore = re.compile(
    r'((?<="duration": )[0-9.]+?(?=,|}))|((?<="code": ").*?(?="))|((?<="id": ").*?(?="))'
)
seek_videore = re.compile(
    r'((?<="code": ").*?(?="))|((?<="new_time": )[0-9.]+?(?=,|}))|((?<="old_time": )[0-9.]+?(?=,|}))|((?<="duration": )[0-9.]+?(?=,|}))|((?<="type": ").*?(?="))|((?<="id": ").*?(?="))'
)
speedchange_videore = re.compile(
    r'((?<="current_time": )[0-9.]+?(?=,|}))|((?<="old_speed": ")[0-9.]+?(?="))|((?<="code": ").*?(?="))|((?<="new_speed": ")[0-9.]+?(?="))|((?<="duration": )[0-9.]+?(?=,|}))|((?<="id": ").*?(?="))'
)


def reduce_to_tuple(groupTuples):
    """
    Reduce an array of tuples to a single tuple
    Eg: [("","","id"),("duration","",""),("","code","")]
        returns ("duration","code","id")
    """
    data = np.array(groupTuples)
    match = pd.DataFrame(data)
    return tuple(match.sum(axis=0))

def video_info_parser(row):
    """
    Returns video id and its duration tuple from event column,
    using only play, pause and stop type events
    """
    etype = row.event_type
    pps = ['play_video', 'pause_video', 'stop_video']
    if etype in pps:
        match = reduce_to_tuple(pps_videore.findall(row.event))
        return match[2], float(match[0])
    elif etype == 'load_video':
        match = reduce_to_tuple(load_videore.findall(row.event))
        return match[2], float(match[0])
    elif etype == 'seek_video':
        match = reduce_to_tuple(seek_videore.findall(row.event))
        return match[5], float(match[3])
    elif etype == 'speed_change_video':
        match = reduce_to_tuple(speedchange_videore.findall(row.event))
        return match[5], float(match[4])
    else:
        return


def generate_video_dataframe(raw_video_data):
    """
    From video type logs, catch videos into it and creates a dataframe
    with its id and duration
    """
    video_cols = ['id', 'duration']
    video_tuples = []
    for index, row in raw_video_data.iterrows():
        video_tuples.append(video_info_parser(row))
    all_video_df = pd.DataFrame(video_tuples,
                                columns=video_cols).drop_duplicates()

    return all_video_df

In [ ]:
videos_df = generate_video_dataframe(video_logs_df) # obtencion de videos con su id y duration

In [ ]:
def video_event_expander(row):
    """
    Returns a tuple with video player info,
    like where user started and stopped the video,
    from event column using only play, pause and stop type events
    """
    etype = row.event_type
    pps = ['play_video', 'pause_video', 'stop_video']
    if etype in pps:
        match = reduce_to_tuple(pps_videore.findall(row.event))
        return (row.username, row.time, row.event_type, match[2],
                float(match[0]), float(match[3]), float(match[3]),
                float(match[3]))

    elif etype == 'load_video':
        match = reduce_to_tuple(load_videore.findall(row.event))
        return (row.username, row.time, row.event_type, match[2],
                float(match[0]), -1, -1, -1)

    elif etype == 'seek_video':
        match = reduce_to_tuple(seek_videore.findall(row.event))
        old = float(match[2])
        new = float(match[1])
        # Seek video forward
        if old < new:
            return (row.username, row.time, 'seek_forward', match[5],
                    float(match[3]), new, old, new)
        # Seek video backward
        elif old > new:
            return (row.username, row.time, 'seek_back', match[5],
                    float(match[3]), new, old, new)
        else:
            return (row.username, row.time, 'seek_equal', match[5],
                    float(match[3]), new, old, new)
    elif etype == 'speed_change_video':
        match = reduce_to_tuple(speedchange_videore.findall(row.event))
        old = float(match[1])
        new = float(match[3])
        if old < new:  # Speed increased
            return (row.username, row.time, 'speed_change_up', match[5],
                    float(match[4]), float(match[0]), old, new)
        elif old > new:  # Speed decreased
            return (row.username, row.time, 'speed_change_down', match[5],
                    float(match[4]), float(match[0]), old, new)
        else:
            return (row.username, row.time, 'speed_change_equal', match[5],
                    float(match[4]), float(match[0]), old, new)

    else:
        return


def expand_event_info(video_logs):
    """
    From video type logs, creates a dataframe
    with expanded video player info,
    initially contained in the event column
    """
    dframe_cols = [
        'username', 'time', 'event_type', 'id', 'duration', 'currenttime',
        'old', 'new'
    ]
    extend_video_tuples = []
    for index, row in video_logs.iterrows():
        extend_video_tuples.append(video_event_expander(row))
    extend_video_logs = pd.DataFrame(extend_video_tuples, columns=dframe_cols)
    return extend_video_logs

In [ ]:
extended_video_logs_df = expand_event_info(video_logs_df) # acciones del usuario sobre un video (reproducir, pausar, adelantar, etc)
# username time event_type id duration currenttime old new

Recuperado de: *back/views/tasks.py*

In [1]:
# funcion original process_views(dataframe)
def get_video_segments_df(raw_video_course_logs):

        extend_video_logs = expand_event_info(raw_video_course_logs)
        sort_video_logs = extend_video_logs[
            extend_video_logs.event_type != 'load_video'].copy() # quitamos load_video pues no aporta informacion de segmentos
        sort_video_logs['time'] = pd.to_datetime(sort_video_logs['time'],
                                                 unit='ns') # ordenamos por tiempo
        sort_video_logs.sort_values('time', inplace=True)
        grouped_logs = sort_video_logs.groupby('username') # ERROR: agrupamos por username
        segments_df = make_segment_dataframe(grouped_logs) # segmentos de video vistos
        return segments_df

In [2]:
video_segments_df = get_video_segments_df(video_logs_df)

NameError: ignored

In [ ]:
# funcion original process_views(dataframe)
def get_video_views_df(dataframe):
        segments_df = get_video_segments_df(dataframe)
        views_df = segments_df[['id', 'username']].drop_duplicates() # visitas a los videos
        return views_df

In [ ]:
views_on_videos_df = get_video_views_df(video_logs_df)

# Cobertura en un video

# Tiempo en un documento

Recuperado de: *back/core/classifier.py*

In [3]:
import re
import json
import pandas as pd
import numpy as np
import logging
import datetime

re_page_view_courseware = re.compile("courseware\/[^/]+([^/]+)*\/?")
re_page_view_course = re.compile("course\/$")
re_wiki_view = re.compile("wiki")
re_progress_view = re.compile("progress$")
re_bookmarks_view = re.compile("bookmarks\/$")
re_page_view_main = re.compile("courses\/.+")
re_page_close = re.compile("^page_close$")
# discussion related
re_forum_view = re.compile("discussion\/forum\/$")
re_forum_view_user_profile = re.compile(
    "discussion\/forum\/users\/[^/]+$")
re_forum_thread_view = re.compile("discussion\/forum\/[^/]+\/threads\/[^/]+$")

# pages for referer
re_from_login = re.compile("eol.uchile.cl\/login?")
re_from_dashboard = re.compile("\/dashboard$")

# navigation
re_next_vertical = re.compile("^seq_next$")
re_next_sequence = re.compile("^edx.ui.lms.sequence.next_selected$")
re_previous_vertical = re.compile("^seq_prev$")
re_previous_sequence = re.compile("^edx.ui.lms.sequence.previous_selected$")
re_goto_vertical = re.compile("^seq_goto$")

logger = logging.getLogger(__name__)


class LogParser:
    """LogParser Class
    Parse logs from edX with type of event and page/vertical id
    """

    def __init__(self, course_filepath="", df=None, run_code=None):
        """LogParser class constructor

        Arguments:
            course_filepath {str} -- file path of the course structure
            df {Dataframe} -- Optional processed pandas dataframe
        """
        self.course_filepath = course_filepath
        if course_filepath == "":
            self.course = df
        else:
            self.course = pd.read_csv(self.course_filepath, sep='\t')
        if run_code is None:
            self.run_code = str(datetime.date.today())
        else:
            self.run_code = run_code

    def debug_course_id(self, id):
        """
        Store course structure and problematic id
        """
        try:
            self.course.to_csv(
                "Course-{}.csv".format(self.course.course_name.iloc[0], self.run_code), index=False)
            with open("Error-ids-{}-{}.log".format(self.course.course_name.iloc[0], self.run_code), 'a') as f:
                f.write(id+'\n')
        except Exception:
            logger.warning("!hola", exc_info=True)

    def load_logs(self, logs):
        """Load DataFrame with logs to parse

        Arguments:
            logs {pandas.core.frame.DataFrame} -- logs to parse
        """
        assert type(logs) == pd.core.frame.DataFrame, \
            'logs must be type pandas.core.frame.DataFrame'

        self.logs = logs.copy().astype({'referer': 'str'})
        self.logs['time'] = pd.to_datetime(self.logs['time'])

    def load_logs_from_file(self, logs_filepath):
        """Load DataFrame with logs to parse from the file

        Arguments:
            logs_filepath {str} -- file path of the logs
        """
        self.load_logs(pd.read_csv(logs_filepath, sep='\t'))

    def parse_event_type(self, event_type):
        """Parse (with regex) the event_type value of a log to classify it

        Arguments:
            event_type {str} -- string to parse/classify

        Returns:
            str -- classification
        """
        if re_page_view_courseware.search(event_type):
            return 'page_view'
        elif re_page_view_course.search(event_type):
            return 'course_view'
        elif re_wiki_view.search(event_type):
            return 'wiki_view'
        elif re_progress_view.search(event_type):
            return 'progress_view'
        elif re_bookmarks_view.search(event_type):
            return 'bookmarks_view'
        elif re_forum_view.search(event_type):
            return 'forum_view'
        elif re_forum_view_user_profile.search(event_type):
            return 'forum_view_user_profile'
        elif re_forum_thread_view.search(event_type):
            return 'forum_thread_view'
        elif re_next_vertical.search(event_type):
            return 'next_vertical'
        elif re_next_sequence.search(event_type):
            return 'next_sequence_first_vertical'
        elif re_previous_vertical.search(event_type):
            return 'previous_vertical'
        elif re_previous_sequence.search(event_type):
            return 'previous_sequence_last_vertical'
        elif re_goto_vertical.search(event_type):
            return 'goto_vertical'
        else:
            return 'not_classified'

    def parse_referer(self, referer):
        """Parse (with regex) the referer value of a log to classify it

        Arguments:
            referer {str} -- string to parse/classify

        Returns:
            str -- classification
        """
        if re_from_dashboard.search(referer):
            return 'dashboard_view'
        elif re_from_login.search(referer):
            return 'login'
        elif re_page_view_courseware.search(referer):
            return 'page_view'
        elif re_page_view_course.search(referer):
            return 'course_view'
        elif re_wiki_view.search(referer):
            return 'wiki_view'
        elif re_progress_view.search(referer):
            return 'progress_view'
        elif re_bookmarks_view.search(referer):
            return 'bookmarks_view'
        elif re_forum_view.search(referer):
            return 'forum_view'
        elif re_forum_view_user_profile.search(referer):
            return 'forum_view_user_profile'
        elif re_forum_thread_view.search(referer):
            return 'forum_thread_view'
        else:
            return 'not_classified'

    def parse_logs(self):
        """Add classification of the event_type and referer value of a log
        and add the vertical id of the page visited on it
        """
        self.logs['classification_event_type'] = self.logs.event_type.apply(
            self.parse_event_type)
        self.logs['classification_referer'] = self.logs.referer.apply(
            self.parse_referer)
        self.logs['event_type_vertical'] = self.logs.apply(
            self.__add_vertical_id, axis=1)

    def parse_and_get_logs(self):
        """Parse and return (parsed) logs

        Returns:
            pandas.core.frame.DataFrame -- logs parsed
        """
        self.parse_logs()
        return self.logs

    def __get_page(self, chapter, sequential, vertical):
        """Gets the row of the course structure DataFrame that matches the
        chapter, sequential and vertical id. If the vertical id is greater
        than the max vertical id, the real value is 1.

        Arguments:
            chapter {str} -- id of the chapter
            sequential {str} -- id of the sequential
            vertical {str} -- id of the vertical

        Returns:
            pandas.core.frame.DataFrame
            -- rows from the course df that matches the condition
        """
        condition = ((self.course.chapter.str.contains(chapter)) &
                     (self.course.sequential.str.contains(sequential)))

        max_vertical = max(self.course[condition]['vertical_number'])
        vertical_number = 1 if int(vertical) > max_vertical else int(vertical)

        condition = ((self.course.chapter.str.contains(chapter)) &
                     (self.course.sequential.str.contains(sequential)) &
                     (self.course.vertical_number == vertical_number))

        return self.course[condition]

    def __add_vertical_id(self, row):
        """Given a row from the logs, adds the vertical id of the page visited according
        to the classification of event_type

        Arguments:
            row {pandas.core.series.Series} -- log to which the id of the vertical is added

        Returns:
            str -- id of the vertical visited in the log
        """
        try:
            cet = row['classification_event_type']
            if cet == 'page_view':
                path = row['event_type'].split('courseware')[1]
                path = path[:-
                            1].split("/") if path[-1] == "/" else path.split("/")
                chapter = sequential = vertical = ''
                chapter = path[1]
                sequential = path[2]
                try:
                    vertical = path[3]
                except IndexError:
                    vertical = '1'
                try:
                    tmp_course = self.__get_page(chapter, sequential, vertical)
                    return tmp_course['vertical'].iloc[0]
                except Exception:
                    # no valid id
                    return "NOT_LISTED"

            elif cet == 'forum_thread_view':
                try:
                    discussion_id = row['event_type'].split('forum')[
                        1].split("/")[1]

                    tmp_course = self.course[self.course.discussion_id ==
                                             discussion_id]

                    return tmp_course['vertical'].iloc[0]
                    # si se quisiera ser mas especifico, cambiar el return por discussion_id
                except AttributeError:
                    return 'DISCUSSION'

            elif cet == 'next_vertical':
                return self.__get_next_vertical(row['event'])

            elif cet == 'next_sequence_first_vertical':
                return self.__get_next_sequence_first_vertical(row['event'])

            elif cet == 'previous_vertical':
                return self.__get_previous_vertical(row['event'])

            elif cet == 'previous_sequence_last_vertical':
                return self.__get_previous_sequence_last_vertical(row['event'])

            elif cet == 'goto_vertical':
                return self.__get_goto_vertical(row['event'])

            elif cet == 'not_classified':
                return 'NOT_CLASSIFIED'

            else:
                # course_view has no classifications yet
                return 'NOT_CLASSIFIED'
        except IndexError:
            event = json.loads(row['event'])['id']
            self.debug_course_id(event)
            logger.warning(
                "Pandas coundn't find index for event: {} - {}".format(cet, event))
            return 'NOT_CLASSIFIED'
        except Exception:
            logger.warning(
                "Unforseen exception, event: {}, skipping row".format(cet))
            return 'NOT_CLASSIFIED'

    def __get_next_vertical(self, event):
        """Given an event from the logs, matches the sequential and next vertical number
        with the data from the course to get the vertical id.

        Arguments:
            event {str} -- event of the event_type seq_next

        Returns:
            str -- vertical id
        """
        # get the next vertical in the same sequential
        json_sequence = json.loads(event)
        sequential = json_sequence['id']
        next_vertical_number = json_sequence['new']
        actual_vertical_number = json_sequence['old']
        assert next_vertical_number == actual_vertical_number + \
            1, 'actual vertical number must be 1 lower than next_vertical_number'

        try:
            row_next_vertical = self.course[((self.course.vertical_number == next_vertical_number) & (
                self.course.sequential == sequential))].iloc[0]
            return row_next_vertical['vertical']
        except IndexError:
            return 'NO_VERTICAL_FOUND'

    def __get_previous_vertical(self, event):
        """Given an event from the logs, matches the sequential and previous vertical number
        with the data from the course to get the vertical id.

        Arguments:
            event {str} -- event of the event_type seq_prev

        Returns:
            str -- vertical id
        """
        # get previous vertical in the same sequential
        json_sequence = json.loads(event)
        sequential = json_sequence['id']
        previous_vertical_number = json_sequence['new']
        actual_vertical_number = json_sequence['old']
        assert previous_vertical_number == actual_vertical_number - \
            1, 'actual vertical number must be 1 greater than previous_vertical_number'
        try:
            row_previous_vertical = self.course[((self.course.vertical_number == previous_vertical_number) & (
                self.course.sequential == sequential))].iloc[0]
            return row_previous_vertical['vertical']
        except IndexError:
            return 'NO_VERTICAL_FOUND'

    def __get_next_sequence_first_vertical(self, event):
        """Given an event from the logs, matches the sequential, gets the first vertical
        of the next sequential and matches it with the data from the course to get its id.
        If the sequential is the last of its chapter, gets the first vertical of the first
        sequential of the next chapter.

        Arguments:
            event {str} -- event of the event_type edx.ui.lms.sequence.next_selected

        Returns:
            str -- vertical id
        """
        # get the first vertical of the next sequential
        json_sequence = json.loads(event)
        sequential = json_sequence['id']
        row = self.course[self.course.sequential == sequential].iloc[0]
        chapter = row['chapter']  # chapter actual
        next_sequential_number = row['sequential_number'] + 1
        try:
            # caso en que hay un sequential mas en ese chapter
            row_next = self.course[((self.course.vertical_number == 1) & (
                self.course.sequential_number == next_sequential_number) & (self.course.chapter == chapter))].iloc[0]
        except IndexError:
            # caso que era el ultimo sequential del chapter
            next_chapter_number = row['chapter_number'] + 1
            row_next = self.course[((self.course.vertical_number == 1) & (
                self.course.sequential_number == 1) & (self.course.chapter_number == next_chapter_number))].iloc[0]
        return row_next['vertical']

    def __get_previous_sequence_last_vertical(self, event):
        """Given an event from the logs, matches the sequential, gets the last vertical
        of the previous sequential and matches it with the data from the course to get its id.
        If the sequential is the first of its chapter, gets the last vertical of the last
        sequential of the previous chapter.

        Arguments:
            event {str} -- event of the event_type edx.ui.lms.sequence.previous_selected

        Returns:
            str -- vertical id
        """
        # get the last vertical of the previous sequential
        json_sequence = json.loads(event)
        sequential = json_sequence['id']
        row = self.course[self.course.sequential == sequential].iloc[0]
        chapter = row['chapter']  # chapter actual
        previous_sequential_number = row['sequential_number'] - 1

        if previous_sequential_number <= 0:
            # maximo vertical y maximo sequential del chapter anterior
            previous_chapter_number = row['chapter_number'] - 1
            previous_chapter = self.course[self.course.chapter_number ==
                                           previous_chapter_number]
            previous_sequential = previous_chapter[previous_chapter.sequential_number ==
                                                   previous_chapter.sequential_number.max()]
        else:
            # buscar el maximo vertical del sequentaial
            previous_sequential = self.course[(
                (self.course.sequential_number == previous_sequential_number) & (self.course.chapter == chapter))]

        last_vertical_previous_sequential = previous_sequential.loc[previous_sequential.vertical_number.idxmax(
        )]
        return last_vertical_previous_sequential['vertical']

    def __get_goto_vertical(self, event):
        """Given an event from the logs, matches the sequential, gets the indicated vertical
        on it and matches it with the data from the course to get its id.

        Arguments:
            event {str} -- event of the event_type seq_goto

        Returns:
            str -- vertical id
        """
        json_sequence = json.loads(event)
        sequential = json_sequence['id']  # id
        goto_vertical_number = json_sequence['new']  # new
        try:
            row_goto_vertical = self.course[((self.course.vertical_number == goto_vertical_number) & (
                self.course.sequential == sequential))].iloc[0]
            return row_goto_vertical['vertical']
        except IndexError:
            return 'NO_VERTICAL_FOUND'

Recuperado de: *back/times/tasks.py*

In [ ]:
def process_times(dataframe, course_df, date, course_id, code):
        course_code = course_df["course"][0]
        # Parse and process time values
        parser = LogParser(df=course_df, run_code=run_code)
        parser.load_logs(dataframe)  # Create log copy and modify
        parsed_logs = parser.parse_and_get_logs()  # Recover log reference
        timer = TimeOnPage(
            timeout_last_action=TIMEOUT,
            timeout_outlier=TIMEOUT,
            navigation_time=COMPUTE_LOWER_LIMIT)
        timer.load_logs(parsed_logs)  # Create log copy and modify
        del parsed_logs  # Free old value
        timer.do_user_time_session()
        timer.do_time_on_page(
            apply_outlier_timeout=True,
            apply_navigation_lower_limit=True)

        # Clean and recover only times on valid verticals
        timer.clean_time_on_page_logs()
        time_per_user_session = timer.time_on_page.copy()
        time_per_user_session = time_per_user_session[
            time_per_user_session['vertical_id'] != 'NOT_LISTED']
        return time_per_user_session


# Visitas a un documento

Recuperado de: *back/visits/tasks.py*

In [ ]:
def compute_visits(dataframe, course_df, date, course_id, code):
        course_code = course_df["course"][0]
        # Parse and process time values
        parser = LogParser(df=course_df, run_code=code)
        parser.load_logs(dataframe)
        parsed_logs = parser.parse_and_get_logs()

        # CLASSIFIER PART
        # Note: we could remove each iteration of indexes
        # to reduce memory usage as we go
        cet = 'classification_event_type'
        condition = (parsed_logs[cet] == 'next_vertical') | \
            (parsed_logs[cet] == 'next_sequence_first_vertical') | \
            (parsed_logs[cet] == 'previous_vertical') | \
            (parsed_logs[cet] == 'previous_sequence_last_vertical') | \
            (parsed_logs[cet] == 'goto_vertical')
        navigation_logs = parsed_logs[condition]
        navigation_logs_cleaned = navigation_logs[
            navigation_logs['event_type_vertical'] != 'NO_VERTICAL_FOUND']

        user_vertical_visits = navigation_logs_cleaned \
            .groupby(['username', 'event_type_vertical']) \
            .event_type_vertical.agg('count') \
            .to_frame('visits') \
            .reset_index()

        merged_sequential_visits = user_vertical_visits \
            .rename(columns={"event_type_vertical": "vertical"}) \
            .merge(course_df, on='vertical')

        with transaction.atomic():
            # Delete today's info to overwrite
            previous_visits = VisitOnPage.objects.filter(
                time__date=date, vertical__course=course_code)
            previous_visits.delete()

            # Upload bulks to DB
            verticals = list(CourseVertical.objects.filter(course=course_code))
            verticals_to_associate = {}
            for b in verticals:
                verticals_to_associate[b.vertical] = b
            count, _ = merged_sequential_visits.shape
            for i in range(0, count, BULK_UPLOAD_SIZE):
                bulk = merged_sequential_visits[i:i+BULK_UPLOAD_SIZE]
                visits = list(bulk.apply(
                    lambda row: make_row(row, date, verticals_to_associate.get(row["vertical"])), axis=1))
                VisitOnPage.objects.bulk_create(visits)

            logger.info("Course {}, {} visits processed for {}".format(
                course_id, count, date))

# Completitud de una unidad